In [1]:
import json
import os.path
import pandas as pd
import numpy as np

dirPath = r'../../../DataSet/KBO/play_ScoreBoard_RawData_json/'

def exportCSV(df, path):
    df.to_csv(path, encoding='utf-8-sig', index=False)

In [2]:
BaseInfo = pd.DataFrame()
ScoreBoard_tbl3 = pd.DataFrame()
Game_ScoreBoard = pd.DataFrame()

## json Parser
BaseInfo
ScoreBoard_tbl3
Game_ScoreBoard

### test Path
test_path = r'../../../DataSet/KBO/play_ScoreBoard_RawData_json/0_2021_20211030WOHT0_GetScoreBoardScroll.json'
jsobj = json.loads(open(test_path, 'r', encoding='utf-8-sig').read())

In [3]:
for fileName in sorted(os.listdir(dirPath)):
    jsobj = json.loads(open(
        os.path.join(dirPath,fileName),
        'r', encoding='utf-8-sig').read())

    ## Baseinfo Parsing
    BaseInfo = BaseInfo.append(pd.json_normalize(jsobj))

    ## table 3 Parsing
    tmp = pd.DataFrame(data=pd.json_normalize(json.loads(jsobj['table3']), record_path=['rows','row'])['Text']).transpose()
    tmp['G_ID'] = jsobj['G_ID']
    ScoreBoard_tbl3 = ScoreBoard_tbl3.append(tmp)

    ## Game_ScoreBoard Parsing
    tmp = pd.json_normalize(pd.json_normalize(json.loads(jsobj['table2']), record_path='rows')['row']).transpose().applymap(lambda x: x['Text'])
    tmp['INNING_CNT'] = pd.json_normalize(json.loads(jsobj['table2']), record_path=['headers', 'row'])['Text']
    tmp.rename(columns={0:'AWAY_SCORE', 1:'HOME_SCORE'}, inplace=True)

    tmp['AWAY_HOME'] = 1

    tmp2 = tmp.copy()
    tmp2['AWAY_HOME'] = 2

    del tmp['HOME_SCORE']
    del tmp2['AWAY_SCORE']

    tmp.rename(columns={'AWAY_SCORE':'SCORE'}, inplace=True)
    tmp2.rename(columns={'HOME_SCORE':'SCORE'}, inplace=True)

    tmp = tmp.append(tmp2)
    tmp['G_ID'] = jsobj['G_ID']

    tmp = tmp[['G_ID', 'INNING_CNT', 'SCORE', 'AWAY_HOME']]
    Game_ScoreBoard = Game_ScoreBoard.append(tmp)

BaseInfo.reset_index(drop=True, inplace=True)
ScoreBoard_tbl3.reset_index(drop=True, inplace=True)
Game_ScoreBoard.reset_index(drop=True, inplace=True)

Game_ScoreBoard.drop(Game_ScoreBoard.loc[Game_ScoreBoard['SCORE'] == '-'].index,inplace=True)

ScoreBoard_tbl3 = ScoreBoard_tbl3[['G_ID',0,1,2,3,4,5,6,7]]
ScoreBoard_tbl3.rename(columns={
    0:'AWAY_R',
    1:'AWAY_H',
    2:'AWAY_E',
    3:'AWAY_B',
    4:'HOME_R',
    5:'HOME_H',
    6:'HOME_E',
    7:'HOME_B'}, inplace=True)

In [4]:
exportCSV(BaseInfo, 'BaseInfo_v2.csv')
exportCSV(ScoreBoard_tbl3, 'ScoreBoard_tbl3_v2.csv')
exportCSV(Game_ScoreBoard, 'Game_ScoreBoard_v2.csv')


# Extra Table
Team_idx
Park_idx
Game_info

In [ ]:
BaseInfo = pd.read_csv('BaseInfo_v2.csv')
ScoreBoard_tbl3 = pd.read_csv('ScoreBoard_tbl3_v2.csv')
Game_ScoreBoard = pd.read_csv('Game_ScoreBoard_v2.csv')

In [5]:
## 팀 목록 # 10개 팀
Team_idx = pd.DataFrame()
i = sorted(BaseInfo['HOME_ID'].unique())
Team_idx['TEAM_CHAR'] = pd.Series(data=i)
Team_idx['TEAM_ID'] = pd.Series(data=list(range(51, 51 + len(i))))

In [6]:
## 경기장 목록 # 13개 구장
Park_idx = pd.DataFrame()
i = sorted(BaseInfo['S_NM'].unique())
Park_idx['S_NM'] = pd.Series(data=i)
Park_idx['PARK_ID'] = pd.Series(data=list(range(11, 11 + len(i))))

In [7]:
## Game_info Feature 구성
# G_ID :: BaseInfo['G_ID']. Game id
# START_TM :: BaseInfo['START_TM']. 경기 시작 시간
# END_TM :: BaseInfo['END_TM']. 경기 종료 시간
# AWAY_HOME :: 홈/어웨이 팀 구분. 어웨이:1, 홈:2
# TEAM_ID :: Team_idx와 join. 팀 코드를 두자리 int로 구성된 ID 값으로 대체.
# SCORE :: 해당 팀의 경기 점수 출력
# PLAY_RESULT :: BaseInfo['T_SCORE_CN','B_SCORE_CN']의 값을 비교하여 승:3, 무:2, 패:1
# PRAK_ID :: BaseInfo['S_NM']과 Park_idx join 수행. asos code와 동일.
# YEAR :: G_ID 앞 4자리

## GameInfo
Game_info = pd.DataFrame()
Game_info['G_ID'] = BaseInfo['G_ID']
Game_info['START_TM'] = BaseInfo['START_TM']
Game_info['END_TM'] = BaseInfo['END_TM']
Game_info['SCORE_CNT'] = BaseInfo['T_SCORE_CN'].apply(str) + ',' + BaseInfo['B_SCORE_CN'].apply(str)

Game_info['S_NM'] = BaseInfo['S_NM']
Game_info = pd.merge(left=Game_info, right=Park_idx, on='S_NM')
del Game_info['S_NM']

# Game_info :: away
# tmp :: home
Game_info['AWAY_HOME'] = np.NaN
tmp = Game_info.copy()
Game_info['AWAY_HOME'] = 1
tmp['AWAY_HOME'] = 2

# score 기록
Game_info['SCORE'] = Game_info['SCORE_CNT'].apply(lambda x: x.split(',')[0])
tmp['SCORE'] = tmp['SCORE_CNT'].apply(lambda x: x.split(',')[1])

# 승3 패2 무1 기록
Game_info['SCORE_CNT'] = Game_info['SCORE_CNT'].apply(lambda x: {x.split(',')[0] > x.split(',')[1]: 3,
                                                                 x.split(',')[0] < x.split(',')[1]: 1}.get(True, 2))
tmp['SCORE_CNT'] = tmp['SCORE_CNT'].apply(lambda x: {x.split(',')[0] > x.split(',')[1]: 1,
                                                     x.split(',')[0] < x.split(',')[1]: 3}.get(True, 2))

# key 역할을 위한
Game_info['TEAM_CHAR'] = Game_info['G_ID'].apply(lambda x: x[-5:-3])
tmp['TEAM_CHAR'] = tmp['G_ID'].apply(lambda x: x[-3:-1])

Game_info = Game_info.append(tmp)

# year 추가
Game_info['YEAR'] = Game_info['G_ID'].apply(lambda x: x[:4])

Game_info = pd.merge(left=Game_info, right=Team_idx, on='TEAM_CHAR')
del Game_info['TEAM_CHAR']

Game_info.rename(columns={'SCORE_CNT':'PLAY_RESULT'}, inplace=True)
Game_info.sort_values(by=['G_ID', 'AWAY_HOME'], ascending=[True, True], ignore_index=True, inplace=True)

In [8]:
# Game_info
# Park_idx
# Team_idx

exportCSV(Team_idx, 'Team_idx.csv')
exportCSV(Park_idx, 'Park_idx.csv')
exportCSV(Game_info, 'Game_info.csv')